# Prompts 101
* https://www.youtube.com/watch?v=REdLBv91I7Q

* https://www.youtube.com/watch?v=J_0qvRt4LNk&t=1s this one is also helpful

# PromptTemplate -> For LLM

## Example 1

In [ ]:
from langchain import PromptTemplate

In [ ]:
simiar_word_template = """

I'm going to give you a word, and you need to give me a related word.

My word: {word}
Your word: 
"""

In [ ]:
prompt = PromptTemplate(
    input_variables = ["word"],
    template=simiar_word_template
    
)

In [5]:
print(prompt)

input_variables=['word'] output_parser=None partial_variables={} template="\n\nI'm going to give you a word, and you need to give me a related word.\n\nMy word: {word}\nYour word: \n" template_format='f-string' validate_template=True


In [6]:
formatted_prompt = prompt.format(word="tree") #insert tree into the bucket

In [7]:
print(formatted_prompt)



I'm going to give you a word, and you need to give me a related word.

My word: tree
Your word: 



## Example 2

In [ ]:
simple_template = "This is my template"

In [ ]:
prompt = PromptTemplate(
    input_variables = [], 
    template = simple_template
)

In [ ]:
formatted_prompt = prompt.format()

In [ ]:
print(formatted_prompt)

## Example 3

In [8]:
complex_template = """
 {name} is {age} years old and enjoys {hobby}.
"""

In [9]:
prompt = PromptTemplate(
    input_variables = ["name", "age", "hobby"],
    template = complex_template
)

In [10]:
formatted_prompt = prompt.format(name="Hao", age="32", hobby="golf")

In [11]:
print(formatted_prompt)


 Hao is 32 years old and enjoys golf.



## Example 4

In [12]:
location_template = """

I live in {city}, {state}.

"""

In [13]:
prompt = PromptTemplate(
    input_variables = ["city", "state"],
    template = location_template
)

In [14]:
print(prompt.input_variables)

['city', 'state']


In [15]:
# another way of pass the template
prompt = PromptTemplate.from_template(location_template)

In [16]:
print(prompt.input_variables)

['city', 'state']


In [17]:
formatted_prompt = prompt.format(city="new york", state="NY")

In [18]:
print(formatted_prompt)



I live in new york, NY.




## Few shot examples

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [ ]:
animal_examples = [
    
    {"animal": "bird", "home": "nest"},
    {"animal": "beaver", "home": "dam"}
]

In [ ]:
example_template = """
Animal: {animal}
Home: {home}
"""

In [ ]:
example_prompt = PromptTemplate(
    input_variables = ["animal", "home"],
    template = example_template
)

In [ ]:
few_shot_prompt = FewShotPromptTemplate(
    examples=animal_examples,
    example_prompt=example_prompt,
    prefix="Idenfity the home for the given animal.",
    suffix="Animal:{animal}\nHome:",
    input_variables=['animal'],
    example_separator="\n"
    
)

In [ ]:
formatted_prompt = few_shot_prompt.format(animal='clown fish')

In [ ]:
print(formatted_prompt)

In [19]:
from langchain.llms import OpenAI

In [20]:
import sys
sys.path.append("..") # Adds higher directory to python module path
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

In [ ]:
llm = OpenAI(model_name="text-davinci-003", temperature=1)

In [ ]:
result = llm(formatted_prompt)

In [ ]:
print(result)

# ChatPromptTemplate -> For ChatModels

In [21]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [22]:
from langchain.chat_models import ChatOpenAI

In [23]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [24]:
sytem_template = """ Help me learn about {subject}. I will tell you what specific topic I want to learn about
"""

In [25]:
system_message_prompt = SystemMessagePromptTemplate.from_template(sytem_template)

In [26]:
human_template = """
I want to learn about {topic}
"""

In [27]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [28]:
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [29]:
print(chat_prompt)

input_variables=['subject', 'topic'] output_parser=None partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], output_parser=None, partial_variables={}, template=' Help me learn about {subject}. I will tell you what specific topic I want to learn about\n', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], output_parser=None, partial_variables={}, template='\nI want to learn about {topic}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [ ]:
messages = chat_prompt.format_prompt(subject="Math", topic ="addition").to_messages()

In [ ]:
print(messages)

In [ ]:
chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

In [ ]:
chat(messages)

# Selectors
https://youtu.be/REdLBv91I7Q?si=ug6-Ey5sTPgp7TQG&t=2072

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [ ]:
# selector checks the maximum lenght it's required and pass the examples

check the similarity selector = https://youtu.be/REdLBv91I7Q?si=wT64LSZGfuGab0bH&t=2807
* embeddings
* chorma
* k= number of example
-> similarity prompt will direclty select the most relevant one based on the sepcificed k value !!!

check the **MaxMarginalRelevanceExampleSelector** = https://youtu.be/REdLBv91I7Q?si=qMFzF5WZkTwFGyeo&t=3144

embeddings
chorma
k= number of example -> will punish the already embedded ones !!!

# Output Parser
https://youtu.be/REdLBv91I7Q?si=7E8SNz0EPAo_qzAp&t=3733

output parser are ways to structure the outputs of the models
1. Pydantic

## Pydantic -> super powerful

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field, validator

In [ ]:
chat = ChatOpenAI(temperature=0)

In [ ]:
class QA(BaseModel):
    question: str = Field(description="the user's questions")
    answer: str = Field(description="the answer to the user's question")
        

    @validator('question')
    def question_ends_with_question_mark(field):
        if field[-1] != '?':
            raise ValueError("Question must end with a question mark")

        return field        

In [ ]:
parser = PydanticOutputParser(pydantic_object=QA)

In [ ]:
user_query = "what is the most popular sport in the world?"

In [ ]:
system_template = "Answer the user's question, \n{format_instructions}\n"

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
human_template = "{question}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [ ]:
messages = chat_prompt.format_prompt(format_instructions=parser.get_format_instructions(), question=user_query).to_messages()

In [ ]:
print(messages[0].content)

In [ ]:
result = chat(messages)
print(result)

In [ ]:
parsed_result = parser.parse(result.content)
print(parsed_result)
print(parsed_result.question)
print(parsed_result.answer)

## Structured Output Parser -> less powerful

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [ ]:
response_schemas = [
    ResponseSchema(name="question", description="the user's questions"),
    ResponseSchema(name="answer", description="the answer to the user's question")
]

In [ ]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
user_query = "what is the most popular sport in the world?"
system_template = "Answer the user's question, \n{format_instructions}\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{question}"
human_message_prompt = HumanMeswgasagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [ ]:
messages = chat_prompt.format_prompt(format_instructions=parser.get_format_instructions(), 
                                     question=user_query).to_messages()
result = chat(messages)
print(result)

In [ ]:
parsed_result = parser.parse(result.content)
print(parsed_result)
print(parsed_result.question) # this is not there 
print(parsed_result.answer) # this is not there

## 3. Comma Seperated List Output Parser -> super simple one

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
parser = CommaSeparatedListOutputParser() # easier setup

In [ ]:
system_template = "name 5 fruits.\n{format_instructions}\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
messages = chat_prompt.format_prompt(format_instructions=parser.get_format_instructions(), 
                                     question=user_query).to_messages()
result = chat(messages)
print(result)

In [ ]:
parsed_result = parser.parse(result.content)
print(parsed_result)

## 4. Output Fixing Parser

In [ ]:
from langchain.output_parsers import PydanticOutputParser,OutputFixingParser
from pydantic import BaseModel, Field, validator

In [ ]:
class QA(BaseModel):
    question: str = Field(description="the user's questions")
    answer: str = Field(description="the answer to the user's question")
        

    @validator('question')
    def question_ends_with_question_mark(field):
        if field[-1] != '?':
            raise ValueError("Question must end with a question mark")

        return field    

In [ ]:
parser = PydanticOutputParser(pydantic_object=QA)

In [ ]:
misformatted = "{'question': what is the most popular sport in the word?, 'answer': 'soccr'}"  #這邊錯誤的原因是沒有"", 這是非常有可能發生的，因為LLM可能會出錯

In [ ]:
# 傳達出正確的錯誤訊息！
parser.parse(misformatted)

In [ ]:
# 這邊就是強迫在叫llm用一次 output parser
fixing_parser = OutputFixingParser.from_llm(llm=chat, parser=parser)

In [ ]:
# 注意看，underlying全部都是prompt... Please try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:
print(fixing_parser)


In [ ]:
fixed_result = fixing_parser.parse(misformatted)

In [ ]:
print(fixed_result)
print(fixed_result.question)
print(fixed_result.answer)